In [1]:
import os
import requests
from ftplib import FTP
import gzip
import io
import pandas as pd
import numpy as np

In [2]:
raw_path = "../data/raw/"

In [20]:
weather_stn = pd.read_pickle('../data/processed/stinson2019/weather_stn')
weather_stn

,stn_name,lat,lon,elevation_m,country,state,start,end
stn_id,,,,,,,,
726358-00384,MICHIGAN CITY MUNICIPAL AIRPO,41.703,-86.821,200.3,US,IN,2012-01-10,2021-03-13
724115-93757,INGALLS FIELD AIRPORT,37.950,-79.817,1156.1,US,VA,2006-01-01,2021-03-13
724117-63802,LONESOME PINE AIRPORT,36.988,-82.530,818.1,US,VA,2006-01-01,2021-03-13
716170-99999,JONQUIERE QUE,48.417,-71.133,128.0,CA,,1992-01-06,2021-03-12
725085-54756,ORANGE MUNICIPAL AIRPORT,42.570,-72.291,169.2,US,MA,1996-07-02,2021-03-13
726116-94765,LEBANON MUNICIPAL AIRPORT,43.626,-72.305,182.3,US,NH,1993-01-01,2021-03-14


In [ ]:
# load a bunch of tables to get required years of data for each site
# append to weather_stn columns sap_data_year (list)

In [22]:
for stn in weather_stn.index: print(stn)

726358-00384
724115-93757
724117-63802
716170-99999
725085-54756
726116-94765


In [ ]:
weather = pd.DataFrame(columns = ["stn_id", "datetime","air_temp"])

for stn in weather_stn.index:
    for year in weather_stn[stn].sap_data_years:

        # Generate filename based on selected station number and year and download
        # data from NOAA FTP site.
        filename = station_number + "-" + str(year) + ".gz"

        compressed_data = io.BytesIO()

        try:
            noaa_ftp = FTP("ftp.ncei.noaa.gov")
            noaa_ftp.login()  # Log in (no user name or password required)
            noaa_ftp.cwd("pub/data/noaa/" + str(year) + "/")
            noaa_ftp.retrbinary("RETR " + filename, compressed_data.write)
        except error_perm as e_message:
            print("Error generated from NOAA FTP site: \n", e_message)
            noaa_ftp.quit()
            return 'FTP Error'

        noaa_ftp.quit()

        # Unzip and process data line by line and extract variables of interest
        # The raw data file format is described here:
        # ftp://ftp.ncei.noaa.gov/pub/data/noaa/isd-format-document.pdf
        
        compressed_data.seek(0)
        stn_year_df = pd.DataFrame(
            columns=[
                "stn_id",
                "datetime",
                "air_temp",

# Additional parameters.  For future use.
#                 "atm_press",
#                 "wind_spd",
#                 "wind_dir",
            ]
        )
        with gzip.open(compressed_data, mode="rt") as stn_data:
            for i, line in enumerate(stn_data):
                stn_year_df.loc[i, "datetime"] = pd.to_datetime(line[15:27])
                stn_year_df.loc[i, "air_temp"] = float(line[87:92]) / 10

        # Additional parameters.  For future use.
        #         stn_year_df.loc[i, "atm_press"] = float(line[99:104]) / 10
        #         stn_year_df.loc[i, "wind_spd"] = float(line[65:69]) / 10
        #         stn_year_df.loc[i, "wind_dir"] = float(line[60:63])

        # Replace missing value indicators with NaNs
        stn_year_df = stn_year_df.replace(
            [999, 999.9, 9999.9], [np.nan, np.nan, np.nan]
        )

        stn_year_df.loc[:, "stn_id"] = station_number
        weather = weather.append(stn_year_df)

weather.set_index("stn_id")

In [3]:
if not os.path.exists(raw_path + '/LEB'):
    os.makedirs(raw_path + '/LEB')

year = "2010"
filename = "726116-94765-2010.gz"
noaa_ftp.cwd("pub/data/noaa/" + year)

In [40]:
os.getcwd()

'/home/steffen/UBC/personal/sapflow/src'

In [42]:
with open(raw_path + '/LEB/' + filename, 'wb+') as stn_gz:
        noaa_ftp.retrbinary('RETR ' + filename, stn_gz.write)

In [45]:
with gzip.open(raw_path + '/LEB/' + filename, 'rb') as stn_data:
    file_content = stn_data.read()


In [48]:
with gzip.open(raw_path + '/LEB/' + filename, mode='rt') as stn_data:
    for i, line in enumerate(stn_data):
        year_df.loc[i, 'datetime'] = pd.to_datetime(line[15:27])
        year_df.loc[i, 'airt'] = float(line[87:92])/10

NameError: name 'year_df' is not defined

Indiana Dunes National Lakeshore - INDU
726358-00384 -- Michigan City Airport

Southernmost Maple
724115-93757 -- Ingalls Field Airport

Divide Ridge
724117-63802 -- Lonesome Pine Airport

Boris
716170-99999 -- Bagotville (CAN-MIL)

Harvard Forest
725085-54756 -- Orange Municipal Airport

Dartmouth Organic Farm
726116-94765 -- Lebanon Municipal Airport 
